# Data Driven Dealings Development


*   EDA on Sales Data
*   RFM Clustering
*   Predicting Sales
*   Market Basket Analysis
*   Recommending Items per Customer







# Reading in the Data

In [1]:
# To be able to use your data stored in your Google Drive you first need to mount your Google Drive so you can load and save files to it. 
from google.colab import drive
drive.mount('/content/gdrive')
#You'll need to put in a token which Google will generate for you as soon as you click on the link

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
data = pd.read_excel('/content/gdrive/My Drive/DDDDFolder/DDDD.xlsx')
data.head()

,SalesDate,SalesValue,SalesAmount,Customer,SalesTransactionID,SalesItem
0,2018-09-28,8280.0,10,0,0,0
1,2018-09-28,7452.0,10,0,0,0
2,2019-04-23,21114.0,30,0,1,0
3,2019-04-23,7038.0,10,0,1,1
4,2019-04-23,7000.0,2,0,1,2


# Sparsity

In [3]:
DataPrep = data[['SalesItem', 'SalesAmount', 'Customer']] #we will only use SalesItem, SalesAmount and Customer for our recommending purpose
DataPrep.head()

,SalesItem,SalesAmount,Customer
0,0,10,0
1,0,10,0
2,0,30,0
3,1,10,0
4,2,2,0


In [4]:
DataPrep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341422 entries, 0 to 341421
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   SalesItem    341422 non-null  int64
 1   SalesAmount  341422 non-null  int64
 2   Customer     341422 non-null  int64
dtypes: int64(3)
memory usage: 7.8 MB


In [5]:
DataGrouped = DataPrep.groupby(['Customer', 'SalesItem']).sum().reset_index() # Group together
DataGrouped.head()



,Customer,SalesItem,SalesAmount
0,0,0,281
1,0,1,158
2,0,2,13
3,0,768,1
4,1,3,2


In [6]:
#make sure that no values <=0 exist
DataGroupedZero = DataGrouped.query('SalesAmount <= 0')
DataGroupedZero.head()

,Customer,SalesItem,SalesAmount


In [7]:
#in our above check we have made sure that no datarows <=0 exists. That is fine!
#only use this in case your data includes values <=0
# DataGrouped.SalesAmount.loc[DataGrouped.SalesAmount == 0] = 1 # Replace a sum of zero purchases with a one to
# DataGrouped.head()

#another interesting way to achieve the same is to use query function
#DataGrouped = DataGrouped.query('SalesAmount > 0') # Only get customers where purchase totals were positive
#DataGrouped.head()


In [8]:
import numpy as np
customers = list(np.sort(DataGrouped.Customer.unique())) # why 36 unique customers in a list and not 35? Index starts at 0!
products = list(DataGrouped.SalesItem.unique()) # Get our unique 3725 unique products that were purchased
quantity = list(DataGrouped.SalesAmount) # All of our purchases
#list function is a list of values. So customers now stores 36 unique customers.

In [9]:
from pandas import DataFrame
DfCustomerUnique = DataFrame(customers,columns=['Customer'])
DfCustomerUnique.head()

,Customer
0,0
1,1
2,2
3,3
4,4


In [10]:
from scipy import sparse
from pandas.api.types import CategoricalDtype

rows = DataGrouped.Customer.astype(CategoricalDtype(categories=customers)).cat.codes # We have got 36 unique customers, which make up 13837 data rows (index)

# Get the associated row indices
cols = DataGrouped.SalesItem.astype(CategoricalDtype(categories= products)).cat.codes # We have got unique 3725 SalesItems, making up 13837 data rows (index)

# Get the associated column indices
#Compressed Sparse Row matrix
PurchaseSparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products))) #len of customers=35, len of products=3725
#csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
#where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]. , see https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

PurchaseSparse
#a sparse matrix is not a pandas dataframe, but sparse matrices are efficient for row slicing and fast matrix vector products


<35x3725 sparse matrix of type '<class 'numpy.longlong'>'
	with 13837 stored elements in Compressed Sparse Row format>

In [11]:
#We have 35 customers with 3725 items. For these user/item interactions, 13837 of these items had a purchase. 
#In terms of sparsity of the matrix, that makes:
MatrixSize = PurchaseSparse.shape[0]*PurchaseSparse.shape[1] # 130375 possible interactions in the matrix (35 unique customers * 3725 unique SalesItems=130375)
PurchaseAmount = len(PurchaseSparse.nonzero()[0]) # 13837 SalesItems interacted with; 
sparsity = 100*(1 - (PurchaseAmount/MatrixSize))
sparsity


89.38676893576223

Since we will use Matrix Factorization for our collaborative filtering it should not be a problem that 89.3% of the interaction matrix is sparse. In plain English, 89,3% in our case means that only 10,7% of our customer-item interactions are already filled, meaning that most items have not been purchased by customers. It is said that collaborative filtering can even work well with even more sparse data. We can prove that it works when checking our decent recommendings in the end. Cosine Similarity is a good measure for sparse data, so we will stick to Cosine (instead of Pearson, Euclidean or Manhattan).

# Recommending

We have already talked about sparsity. However, we will start with a simple recommender first, before we come to more advanced techniques also using optimization for sparse matrices. However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [12]:
#for every dataset we will add a 1 as purchased. That means, that this customer has purchased this item, no matter how many. We use this binary data for our recommending. Another approach would be to use the SalesAmount and 
#normalize it, in case you want to treat the Amount of SalesItems purchased as a kind of taste factor, meaning that someone who bought SalesItem x 100 times, while another Customer bought that same SalesItem x only 5 times does 
#not like it as much. I believe, that very often in Sales a binary approach makes more sense, but of course that depends on the data.
def create_DataBinary(DataGrouped):
    DataBinary = DataPrep.copy()
    DataBinary['PurchasedYes'] = 1 
    return DataBinary

DataBinary = create_DataBinary(DataGrouped)
DataBinary.head()



,SalesItem,SalesAmount,Customer,PurchasedYes
0,0,10,0,1
1,0,10,0,1
2,0,30,0,1
3,1,10,0,1
4,2,2,0,1


In [13]:
data2=DataBinary.drop(['SalesAmount'], axis=1)
data2.head()

,SalesItem,Customer,PurchasedYes
0,0,0,1
1,0,0,1
2,0,0,1
3,1,0,1
4,2,0,1


In [14]:
#for better convenience we add I for Item for every SalesItem. Otherwise we would only have customer and SalesItem Numbers, which can be a little bit puzzling.
data2['SalesItem'] = 'I' + data2['SalesItem'].astype(str)

In [15]:

#DfMatrix = pd.pivot_table(data,index=["Customer"], columns='SalesItem')
DfMatrix = pd.pivot_table(data2, values='PurchasedYes', index='Customer', columns='SalesItem')
DfMatrix.head()

SalesItem,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,I1032,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
Customer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,...,1.0,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#since we are only using 1 and 0, we do not need to think about normalization. But talk is cheap, let`s check to see that even if we would normalize, the result is the same, of course:
DfMatrix=DfMatrix.fillna(0) #NaN values need to get replaced by 0, meaning they have not been purchased yet.
DfMatrixNorm3 = (DfMatrix-DfMatrix.min())/(DfMatrix.max()-DfMatrix.min())
DfMatrixNorm3.head()
#the proof is in the pudding. But we will come back to normalization later on again, when we will take real Sales Amount into consideration for recommending as well.

SalesItem,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,I1032,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
Customer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#we need to bring our pivot table into the desired format, via reset_index and rename_axis. 
DfResetted = DfMatrix.reset_index().rename_axis(None, axis=1) 
DfResetted.head()
#Now each row represents one customer`s buying behaviour: 1 means the customer has purchased, NaN the customer has not yet purchased it

,Customer,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:

DfMatrix.shape

(35, 3725)

In [19]:
df=DfResetted #now working: because Customer must be nvarchar! If customer is int, then failure during CustItemSimilarity!

In [20]:
#we need to replace the NaN values with a 0, because our function will not work on NaN values.
#Please note, that we are only checking if a specific customer bought a specific item, yes or no. That is called binary. If customer bought a specific item, that means 1. If not, then 0. Because of this binary problem there is 
#no use in using any further scaling techniques.
df=df.fillna(0)
df.head()

,Customer,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#Creating a dataframe which only includes Sales Items. Customer is indexed instead.
DfSalesItem = df.drop('Customer', 1) 
DfSalesItem.head()

,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,I1032,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#Calculate the Item based recommendation
import numpy as np
# We will normalize dataframe now, due to ..
#I believe we do not need to normalize, but let us compare..
#vectorized
DfSalesItemNorm = DfSalesItem / np.sqrt(np.square(DfSalesItem).sum(axis=0)) 
DfSalesItemNorm.head()

,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,I1032,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
0,0.333333,0.301511,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,0.000000,0.000000,0.408248,0.288675,0.5,0.0,0.0,0.0,0.0,0.333333,0.288675,0.0,0.0,0.0,0.27735,0.0,0.0,0.0,0.0,0.408248,0.0,0.0,0.0,0.353553,0.353553,0.301511,0.0,0.447214,0.353553,0.0,0.408248,0.447214,0.0,0.377964,0.447214,0.0,0.267261,0.408248,0.408248,0.0,...,0.377964,0.000000,0.333333,0.0,0.377964,0.0,0.333333,0.316228,0.408248,0.353553,0.0,0.353553,0.316228,0.5,0.0,0.0,0.0,0.447214,0.288675,0.000000,0.0,0.447214,0.447214,0.0,0.377964,0.0,0.0,0.333333,0.0,0.288675,0.0,0.447214,0.447214,0.0,0.377964,0.0,0.0,0.353553,0.408248,0.0
3,0.000000,0.000000,0.408248,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.27735,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.267261,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.288675,0.0,0.0,0.0,0.0,0.0,0.000000,0.288675,0.0,0.0,0.0,0.27735,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.353553,0.301511,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.267261,0.000000,0.000000,0.0,...,0.000000,0.408248,0.333333,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.288675,0.377964,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.288675,0.0,0.447214,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [23]:
# Calculating with Vectors to compute Cosine Similarities
ItemItemSim = DfSalesItemNorm.transpose().dot(DfSalesItemNorm) 
ItemItemSim.head()

,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,I1032,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
I0,1.000000,0.703526,0.136083,0.192450,0.333333,0.384900,0.235702,0.333333,0.384900,0.222222,0.192450,0.272166,0.235702,0.235702,0.184900,0.333333,0.272166,0.272166,0.298142,0.272166,0.298142,0.298142,0.298142,0.235702,0.235702,0.201008,0.272166,0.298142,0.235702,0.235702,0.272166,0.298142,0.298142,0.251976,0.298142,0.272166,0.178174,0.272166,0.272166,0.298142,...,0.251976,0.272166,0.222222,0.251976,0.251976,0.251976,0.222222,0.210819,0.136083,0.235702,0.384900,0.235702,0.210819,0.333333,0.384900,0.235702,0.384900,0.298142,0.192450,0.251976,0.384900,0.149071,0.149071,0.235702,0.251976,0.166667,0.298142,0.222222,0.333333,0.192450,0.333333,0.298142,0.298142,0.192450,0.251976,0.384900,0.384900,0.235702,0.272166,0.384900
I1,0.703526,1.000000,0.123091,0.174078,0.150756,0.174078,0.000000,0.301511,0.174078,0.201008,0.174078,0.123091,0.000000,0.000000,0.167248,0.150756,0.246183,0.246183,0.269680,0.123091,0.134840,0.134840,0.134840,0.213201,0.106600,0.090909,0.123091,0.134840,0.213201,0.213201,0.246183,0.134840,0.134840,0.227921,0.134840,0.246183,0.161165,0.246183,0.246183,0.269680,...,0.227921,0.123091,0.201008,0.227921,0.227921,0.113961,0.201008,0.190693,0.123091,0.106600,0.174078,0.106600,0.190693,0.150756,0.174078,0.213201,0.174078,0.134840,0.174078,0.113961,0.174078,0.134840,0.134840,0.213201,0.227921,0.150756,0.134840,0.201008,0.301511,0.174078,0.301511,0.134840,0.134840,0.174078,0.227921,0.174078,0.174078,0.213201,0.246183,0.174078
I10,0.136083,0.123091,1.000000,0.589256,0.408248,0.235702,0.000000,0.204124,0.235702,0.680414,0.589256,0.333333,0.000000,0.000000,0.679366,0.408248,0.500000,0.500000,0.365148,0.666667,0.547723,0.547723,0.547723,0.577350,0.577350,0.615457,0.666667,0.547723,0.577350,0.288675,0.500000,0.547723,0.547723,0.617213,0.547723,0.500000,0.654654,0.500000,0.500000,0.365148,...,0.462910,0.500000,0.544331,0.462910,0.462910,0.617213,0.680414,0.645497,0.833333,0.721688,0.235702,0.721688,0.645497,0.408248,0.235702,0.288675,0.235702,0.547723,0.589256,0.462910,0.235702,0.547723,0.547723,0.288675,0.617213,0.612372,0.365148,0.680414,0.204124,0.589256,0.204124,0.365148,0.547723,0.235702,0.462910,0.235702,0.235702,0.577350,0.500000,0.235702
I100,0.192450,0.174078,0.589256,1.000000,0.577350,0.500000,0.408248,0.577350,0.500000,0.866025,0.833333,0.589256,0.408248,0.408248,0.960769,0.577350,0.707107,0.707107,0.645497,0.707107,0.645497,0.645497,0.645497,0.714435,0.816497,0.957427,0.707107,0.645497,0.714435,0.408248,0.707107,0.645497,0.645497,0.763763,0.645497,0.707107,0.925820,0.707107,0.707107,0.645497,...,0.654654,0.707107,0.866025,0.654654,0.654654,0.763763,0.769800,0.912871,0.707107,0.816497,0.500000,0.816497,0.821584,0.577350,0.500000,0.408248,0.500000,0.645497,1.000000,0.763763,0.500000,0.645497,0.645497,0.408248,0.763763,0.577350,0.516398,0.866025,0.577350,0.916667,0.577350,0.645497,0.645497,0.500000,0.763763,0.500000,0.500000,0.714435,0.707107,0.500000
I1000,0.333333,0.150756,0.408248,0.577350,1.000000,0.866025,0.707107,0.750000,0.866025,0.666667,0.577350,0.612372,0.707107,0.707107,0.554700,0.750000,0.612372,0.612372,0.670820,0.816497,0.670820,0.670820,0.670820,0.707107,0.707107,0.603023,0.612372,0.894427,0.707107,0.707107,0.816497,0.894427,0.670820,0.755929,0.894427,0.612372,0.534522,0.816497,0.816497,0.670820,...,0.755929,0.612372,0.666667,0.566947,0.755929,0.566947,0.666667,0.632456,0.612372,0.707107,0.866025,0.707107,0.632456,1.000000,0.866025,0.707107,0.866025,0.894427,0.577350,0.566947,0.866025,0.670820,0.670820,0.707107,0.755929,0.500000,0.670820,0.666667,0.750000,0.577350,0.750000,0.894427,0

In [24]:
#Another approach to the above would be using corr fct
#Where is the difference?
SalesItemCorrelation = DfSalesItem.corr()
SalesItemCorrelation.head()

,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,I1032,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
I0,1.000000,0.587412,-0.094161,-0.149528,0.199600,0.286896,0.136795,0.199600,0.286896,-0.047009,-0.149528,0.079293,0.136795,0.136795,-0.181680,0.199600,0.079293,0.079293,0.133440,0.079293,0.133440,0.133440,0.133440,-0.008896,-0.008896,-0.116678,0.079293,0.133440,-0.008896,0.136795,0.079293,0.133440,0.133440,0.032686,0.133440,0.079293,-0.213504,0.079293,0.079293,0.133440,...,0.032686,0.079293,-0.047009,0.032686,0.032686,0.032686,-0.047009,-0.082690,-0.094161,-0.008896,0.286896,-0.008896,-0.082690,0.199600,0.286896,0.136795,0.286896,0.133440,-0.149528,0.032686,0.286896,-0.053376,-0.053376,0.136795,0.032686,-0.005871,0.133440,-0.047009,0.199600,-0.149528,0.199600,0.133440,0.133440,0.053376,0.032686,0.286896,0.286896,-0.008896,0.079293,0.286896
I1,0.587412,1.000000,-0.144639,-0.229686,-0.049743,0.012563,-0.166667,0.143701,0.012563,-0.116678,-0.229686,-0.144639,-0.166667,-0.166667,-0.265667,-0.049743,0.018663,0.018663,0.075378,-0.144639,-0.100504,-0.100504,-0.100504,-0.075378,-0.221946,-0.325758,-0.144639,-0.100504,-0.075378,0.098485,0.018663,-0.100504,-0.100504,-0.030773,-0.100504,0.018663,-0.301511,0.018663,0.018663,0.075378,...,-0.030773,-0.144639,-0.116678,-0.030773,-0.030773,-0.184637,-0.116678,-0.155700,-0.144639,-0.221946,0.012563,-0.221946,-0.155700,-0.049743,0.012563,0.098485,0.012563,-0.100504,-0.229686,-0.184637,0.012563,-0.100504,-0.100504,0.098485,-0.030773,-0.049743,-0.100504,-0.116678,0.143701,-0.229686,0.143701,-0.100504,-0.100504,0.012563,-0.030773,0.012563,0.012563,-0.075378,0.018663,0.012563
I10,-0.094161,-0.144639,1.000000,0.470011,0.313164,0.131534,-0.111979,0.074887,0.131534,0.599657,0.470011,0.195402,-0.111979,-0.111979,0.591720,0.313164,0.396552,0.396552,0.247594,0.597701,0.464238,0.464238,0.464238,0.474555,0.474555,0.508569,0.597701,0.464238,0.474555,0.214625,0.396552,0.464238,0.464238,0.530669,0.464238,0.396552,0.557086,0.396552,0.396552,0.247594,...,0.341144,0.396552,0.426202,0.341144,0.341144,0.530669,0.599657,0.551383,0.798851,0.655092,0.131534,0.655092,0.551383,0.313164,0.131534,0.214625,0.131534,0.464238,0.470011,0.341144,0.131534,0.464238,0.464238,0.214625,0.530669,0.551442,0.247594,0.599657,0.074887,0.470011,0.074887,0.247594,0.464238,0.131534,0.341144,0.131534,0.131534,0.474555,0.396552,0.131534
I100,-0.149528,-0.229686,0.470011,1.000000,0.497305,0.423896,0.340825,0.497305,0.423896,0.814531,0.746377,0.470011,0.340825,0.340825,0.939651,0.497305,0.629724,0.629724,0.565194,0.629724,0.565194,0.565194,0.565194,0.610246,0.753592,0.937268,0.629724,0.565194,0.610246,0.340825,0.629724,0.565194,0.565194,0.692219,0.565194,0.629724,0.884652,0.629724,0.629724,0.565194,...,0.541736,0.629724,0.814531,0.541736,0.541736,0.692219,0.676809,0.875595,0.629724,0.753592,0.423896,0.753592,0.742352,0.497305,0.423896,0.340825,0.423896,0.565194,1.000000,0.692219,0.423896,0.565194,0.565194,0.340825,0.692219,0.497305,0.393179,0.814531,0.497305,0.873188,0.497305,0.565194,0.565194,0.423896,0.692219,0.423896,0.423896,0.610246,0.629724,0.423896
I1000,0.199600,-0.049743,0.313164,0.497305,1.000000,0.852386,0.685344,0.717742,0.852386,0.610541,0.497305,0.551442,0.685344,0.685344,0.467293,0.717742,0.551442,0.551442,0.623250,0.789719,0.623250,0.623250,0.623250,0.659912,0.659912,0.530589,0.551442,0.879883,0.659912,0.685344,0.789719,0.879883,0.623250,0.718421,0.879883,0.551442,0.439941,0.789719,0.789719,0.623250,...,0.718421,0.551442,0.610541,0.493915,0.718421,0.493915,0.610541,0.567962,0.551442,0.659912,0.852386,0.659912,0.567962,1.000000,0.852386,0.685344,0.852386,0.879883,0.497305,0.493915,0.852386,0.623250,0.6232

In [25]:
#ItemItemSim.to_excel("ExportItem-Item.xlsx")
# Create a placeholder items for closes neighbours to an item
ItemNeighbours = pd.DataFrame(index=ItemItemSim.columns,columns=range(1,10))
ItemNeighbours.head()

,1,2,3,4,5,6,7,8,9
I0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Create a placeholder items for closes neighbours to an item
#ItemNeighbours = pd.DataFrame(index=ItemItemSim.columns,columns=range(1,10)) 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(ItemItemSim.columns)):
    ItemNeighbours.iloc[i,:9] = ItemItemSim.iloc[0:,i].sort_values(ascending=False)[:9].index
    #we only have 9 items, so we can max recommend 9 items (itself included)
 


In [27]:
ItemNeighbours.head()


,1,2,3,4,5,6,7,8,9
I0,I0,I2,I1,I769,I1134,I705,I704,I1139,I1138
I1,I1,I768,I759,I758,I754,I757,I749,I750,I753
I10,I10,I1699,I1696,I1674,I2102,I19,I1242,I970,I254
I100,I161,I86,I146,I128,I71,I152,I193,I89,I200
I1000,I747,I962,I1041,I893,I930,I1000,I790,I975,I917


In [28]:
ItemNeighbours.head().iloc[:11,1:9]
#it needs to start at position 1, because position 0 is itself

,2,3,4,5,6,7,8,9
I0,I2,I1,I769,I1134,I705,I704,I1139,I1138
I1,I768,I759,I758,I754,I757,I749,I750,I753
I10,I1699,I1696,I1674,I2102,I19,I1242,I970,I254
I100,I86,I146,I128,I71,I152,I193,I89,I200
I1000,I962,I1041,I893,I930,I1000,I790,I975,I917


In [29]:
ItemNeighbours.to_excel("ExportItem-Item-data_neighbours.xlsx")

Now we will create a customer based recommendation which we need our item similarity matrix for. Then we will have a look which items our customers have bought and get the top N neighbours for each item. Afterwards we calculate the purchase history of the customer for each neighbour and calculate a similarity score for them. So in the end we just have to recommend the items with the highest score. 

In [30]:
#Now we will build a Customer based recommendation, which is build upon the item-item similarity matrix, which we have just calculated above.
# Create a place holder matrix for similarities, and fill in the customer column
CustItemSimilarity = pd.DataFrame(index=df.index,columns=df.columns)
CustItemSimilarity.iloc[:,:1] = df.iloc[:,:1]

In [31]:
CustItemSimilarity.head()

,Customer,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Getting the similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities) 

In [33]:
# This takes ages (35 customers * 3725 items)
#We now loop through the rows and columns filling in empty spaces with similarity scores.
#Note that we score items that the customer has already consumed as 0, because there is no point recommending it again.
from timeit import default_timer as timer #to see how long the computation will take
start = timer()


for i in range(0,len(CustItemSimilarity.index)):
    for j in range(1,len(CustItemSimilarity.columns)):
        user = CustItemSimilarity.index[i]
        product = CustItemSimilarity.columns[j]
 
        if df.loc[i][j] == 1:
            CustItemSimilarity.loc[i][j] = 0
        else:
            ItemTop = ItemNeighbours.loc[product][1:9] #
            #do not use order but sort_values in latest pandas
            ItemTopSimilarity = ItemItemSim.loc[product].sort_values(ascending=False)[1:9]
            #here we will use the item dataframe, which we generated during item-item matrix 
            CustomerPurchasings = DfSalesItem.loc[user,ItemTop]
 
            CustItemSimilarity.loc[i][j] = getScore(CustomerPurchasings,ItemTopSimilarity)

end = timer()

print('\nRuntime: %0.2fs' % (end - start))

#if there occurs a strange error  tz=getattr(series.dtype, 'tz', None) .. pandas index.. then this might be if you have used int
# as column headers instead of string


Runtime: 450.15s


In [34]:
CustItemSimilarity.head()

,Customer,I0,I1,I10,I100,I1000,I1001,I1002,I1003,I1004,I1005,I1006,I1007,I1008,I1009,I101,I1010,I1011,I1012,I1013,I1014,I1015,I1016,I1017,I1018,I1019,I102,I1020,I1021,I1022,I1023,I1024,I1025,I1026,I1027,I1028,I1029,I103,I1030,I1031,...,I963,I964,I965,I966,I967,I968,I969,I97,I970,I971,I972,I973,I974,I975,I976,I977,I978,I979,I98,I980,I981,I982,I983,I984,I985,I986,I987,I988,I989,I99,I990,I991,I992,I993,I994,I995,I996,I997,I998,I999
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0.121708,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.122838,0,0.121708,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0.114834,0,0,0,0.123409,0,0,0,0.114834,0.242708,0.242708,0,0,0.247053,0,0,0,0,0,0,0,0,0,0,0,0.247053,0,0,0.242708,0,0,0,...,0,0,0,0.743582,0,0.867513,0,0,0,0,0,0,0,0,0,0,0,0,0,0.247861,0,0,0,0,0,0,0,0,0.114834,0,0.114834,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0.128592,0,0,0,0,0,0,0,0,0,0,0,0,0.121845,0,0.24324,0,0,0,0,0,0,0,0.12393,0,0,0,0,0,...,0,0,0,0,0,0,0,0.1213,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.12393,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0.122612,0,0,0,0,0,0,0.365125,0,0.123409,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.121767,0.117963,0,0,0,0.117963,0,0,0.117963,0,0,0,0,...,0,0,0,0,0,0,0,0.242599,0,0,0,0,0.12243,0,0,0,0,0.117963,0,0,0,0.11758,0.11758,0,0,0,0,0.365125,0,0,0,0,0.117963,0,0.495722,0,0,0,0,0


In [35]:
#now generate a matrix of customer based recommendations
CustItemRecommend = pd.DataFrame(index=CustItemSimilarity.index, columns=['Customer','1','2','3','4','5','6']) #Top 1,2..6
CustItemRecommend.head()

,Customer,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
CustItemRecommend.iloc[0:,0] = CustItemSimilarity.iloc[:,0]
CustItemRecommend.head()

,Customer,1,2,3,4,5,6
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#Instead of having the matrix filled with similarity scores we want to see the product names.
for i in range(0,len(CustItemSimilarity.index)):
    CustItemRecommend.iloc[i,1:] = CustItemSimilarity.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose()

In [38]:
CustItemRecommend.head()


,Customer,1,2,3,4,5,6
0,0,I1134,I999,I2128,I2126,I2125,I2124
1,1,I1194,I650,I1133,I1132,I292,I408
2,2,I1165,I168,I169,I272,I299,I394
3,3,I192,I92,I61,I73,I108,I229
4,4,I1165,I280,I1179,I157,I6,I124


In [39]:

CustItemRecommend.to_excel("ExportCustomer-Item-CustItemRecommend.xlsx")
#We have coded a binary recommender engine, which works only sufficient on a smal data set. Let us see in the next chapter if we can enhance the performance and scalability.

# A more performant approach
# sklearn.metrics.pairwise cosine_similarity

In [40]:
import pandas as pd
import numpy as np
#We will use optimized recommender libraries instead of hand coding the functions like we did in the previous chapter. This hopefully brings us convenience and performance boost
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
#data = pd.read_excel('/content/gdrive/My Drive/DDDDFolder/DDDD.xlsx') 
def create_DataBinary(data):
    DataBinary = data.copy()
    DataBinary['PurchasedYes'] = 1
    return DataBinary

DataBinary = create_DataBinary(data)
data=DataBinary.drop(['SalesValue','SalesDate','SalesTransactionID','SalesAmount'], axis=1)
DfMatrix = pd.pivot_table(data, values='PurchasedYes', index='Customer', columns='SalesItem')
DfResetted = DfMatrix.reset_index().rename_axis(None, axis=1)
DfResetted=DfResetted.fillna(0)
data=DfResetted
data_items = data.drop('Customer', 1)
#Starting point now is the same like in our previous approach:

In [41]:
#Compute Item-Item cosine similarity
# As a first step we normalize the user vectors to unit vectors.
# Cosine similarity, or the cosine kernel, computes similarity as the normalized dot product of X and Y:
#  K(X, Y) =  X, Y  / (||X||*||Y||)
# magnitude = sqrt(x2 + y2 + z2 + ...)
magnitude = np.sqrt(np.square(data_items).sum(axis=1))
# unitvector = (x / magnitude, y / magnitude, z / magnitude, ...)
data_items = data_items.divide(magnitude, axis='index')
#Compute the column-wise cosine similarity using our sparse matrix
def GetItemItemSim(data_items):
    data_sparse = sparse.csr_matrix(data_items)
    #SalesItemCustomerMatrixs=csr_matrix(([1]*len(user_ids),(product_ids, user_ids))) #hier wird das data_items also selbst aufgebaut
    #no difference, no matter if you use data_sparse or not?!--> does normalizing make the differnce?
    #similarities = cosine_similarity(data_items.transpose())
    similarities = cosine_similarity(data_sparse.transpose())#warum transpose?
    #similarity=cosine_similarity(SalesItemCustomerMatrixs)#das gleiche, aber ohne transpose-wechselt Zeilen/Spalten
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    #Return a Pandas DataFrame Matrix including the Similarities
    return sim #why not ,similarities?

data_matrix = GetItemItemSim(data_items)
#Compute Customer-Item Matrix to store which SalesItems the Customer has bought.
Customer = 0 # The id of the user for whom we want to generate recommendations --> change!
CustomerIndex = data[data.Customer == Customer].index.tolist()[0] # Get the frame index
CustomerItemPurch = data_items.iloc[CustomerIndex]
CustomerItemPurch = CustomerItemPurch[CustomerItemPurch >0].index.values
# Users likes for all items as a sparse vector.
user_rating_vector = data_items.iloc[CustomerIndex]
# Calculate the score.
score = data_matrix.dot(user_rating_vector).div(data_matrix.sum(axis=1))
# Remove the known likes from the recommendation.
score = score.drop(CustomerItemPurch)

#Customer Item Calculation

# Construct a new dataframe with the 10 closest neighbours (most similar) for each Customer
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
for i in range(0, len(data_matrix.columns)):
    data_neighbours.iloc[i,:9] = data_matrix.iloc[0:,i].sort_values(ascending=False)[:9].index  #nearest 9 neighbours?

# Construct the neighbourhood from the most similar SalesItems to the ones the Customer has already liked.
most_similar_to_likes = data_neighbours.iloc[CustomerItemPurch]
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
#I did just drop nan, but I need to check this since it might result in wrong recommendations
similar_list = [similar_list for similar_list in similar_list if str(similar_list) != 'nan']

neighbourhood = data_matrix[similar_list].iloc[similar_list]
# A Customer vector containing only the neighbourhood SalesItems and the known Customer likes.
user_vector = data_items.iloc[CustomerIndex].iloc[similar_list]
# Calculate the score.
score = neighbourhood.dot(user_vector).div(neighbourhood.sum(axis=1))
# Drop the known likes.
score = score.drop(CustomerItemPurch)

In [42]:
data_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3695,3696,3697,3698,3699,3700,3701,3702,3703,3704,3720,3721,3722,3723,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3750
0,1.000000,0.705107,0.916830,0.000000,0.001032,0.004268,0.004551,0.003455,0.003455,0.003455,0.002024,0.004256,0.004256,0.003395,0.003904,0.004130,0.004066,0.003455,0.003455,0.004256,0.000854,0.003892,0.003844,0.007554,0.006806,0.006806,0.007523,0.004444,0.009022,0.009022,0.009022,0.009022,0.009022,0.009022,0.009022,0.009022,0.007554,0.008262,0.008262,0.005944,...,0.015827,0.015827,0.015827,0.015827,0.015827,0.015827,0.015827,0.015827,0.015827,0.015827,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.705107,1.000000,0.565765,0.000000,0.005755,0.002243,0.011378,0.001816,0.001816,0.001816,0.002372,0.002236,0.002236,0.008488,0.009760,0.002170,0.010165,0.001816,0.001816,0.002236,0.002134,0.004559,0.009612,0.018887,0.017017,0.017017,0.018809,0.000000,0.004740,0.004740,0.004740,0.004740,0.004740,0.004740,0.004740,0.004740,0.018887,0.004341,0.004341,0.014860,...,0.018541,0.018541,0.018541,0.018541,0.018541,0.018541,0.018541,0.018541,0.018541,0.018541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.916830,0.565765,1.000000,0.000000,0.001126,0.002088,0.002226,0.001690,0.001690,0.001690,0.002208,0.002082,0.002082,0.001661,0.001910,0.002020,0.001989,0.001690,0.001690,0.002082,0.000418,0.004245,0.001881,0.003695,0.003330,0.003330,0.003680,0.000000,0.004413,0.004413,0.004413,0.004413,0.004413,0.004413,0.004413,0.004413,0.003695,0.004042,0.004042,0.002908,...,0.017263,0.017263,0.017263,0.017263,0.017263,0.017263,0.017263,0.017263,0.017263,0.017263,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.000000,1.000000,0.932249,0.507131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.456764,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.001032,0.005755,0.001126,0.932249,1.000000,0.529529,0.204978,0.172531,0.172531,0.172531,0.095683,0.140506,0.140506,0.193088,0.186835,0.131156,0.168804,0.172531,0.172531,0.140585,0.048549,0.218063,0.533938,0.105035,0.121240,0.121240,0.066170,0.072662,0.016675,0.016675,0.016675,0.016675,0.016675,0.016675,0.016675,0.016675,0.105035,0.015271,0.015271,0.143553,...,0.065224,0.065224,0.065224,0.065224,0.065224,0.065224,0.065224,0.065224,0.065224,0.065224,0.165048,0.165048,0.120794,0.120794,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.134852,0.0,0.0,0.0,0.0,0.0,0.0,0.134852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [43]:
# Lets get the top 10 similar items for item 0; only works, if items are int, not navarchar
print (data_matrix.loc[0].nlargest(10)) 

0       1.000000
2       0.916830
1       0.705107
768     0.533081
704     0.442185
769     0.423981
705     0.399591
706     0.397427
1134    0.290744
1135    0.252792
Name: 0, dtype: float64


In [44]:
user_rating_vector.head()

0    0.5
1    0.5
2    0.5
3    0.0
4    0.0
Name: 0, dtype: float64

In [45]:
score.head()

704     0.101308
769     0.180301
705     0.114797
706     0.114599
1134    0.066635
dtype: float64

In [46]:
# Print the known likes and the top 20 recommendations.
print (CustomerItemPurch)
print (score.nlargest(20))
#known customer likes are 0 and 2 and their most recommended items are:

[0 1 2 768]
769     0.180301
705     0.114797
706     0.114599
704     0.101308
758     0.079916
759     0.079916
701     0.078162
753     0.074658
756     0.074496
1134    0.066635
1135    0.063053
1138    0.062198
1139    0.062198
dtype: float64


In [47]:
print (CustomerItemPurch)
print (score.nlargest(6))

[0 1 2 768]
769    0.180301
705    0.114797
706    0.114599
704    0.101308
758    0.079916
759    0.079916
dtype: float64


# Pandas and Scikit-Learn Stack

In [48]:
# Now let's see if we can improve the performance even further
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from timeit import default_timer as timer

In [49]:
#df = pd.read_excel('/content/gdrive/My Drive/DDDDFolder/DDDD.xlsx') #now working: because Customer must be nvarchar! If customer is int, then failure during CustItemSimilarity!
purchase_data=data2
purchase_data.head()

,SalesItem,Customer,PurchasedYes
0,I0,0,1
1,I0,0,1
2,I0,0,1
3,I1,0,1
4,I2,0,1


In [50]:
#Compute Item-Item cosine similarity
#Customer list-like, meaning Customer at n-th position of list purchased n-th SalesItem
#Salesitem also list-like, meaning SalesItem at n-th position of list purchased by n-th Customer
#Returning an Item-Item Similarity Matrix (array-like)
def GetItemItemSim(user_ids, product_ids):
    SalesItemCustomerMatrixs = csr_matrix(([1]*len(user_ids), (product_ids, user_ids))) 
    similarity = cosine_similarity(SalesItemCustomerMatrixs)
    return similarity, SalesItemCustomerMatrixs

#check=GetItemItemSim(user_ids, product_ids)

In [51]:
#Compute Top SalesItem recommendations per Customer
#using the Item-Item Similarity Matrix from above cell
#creating a SalesItemCustomerMatrixs which is also an array, meaning SalesItems per rows and Customer as columns as a binary incidence matrix
#Top_n can be set by yourself, must be int only
#Returning recommendations per Customer as a Pandas DataFrame
def get_recommendations_from_similarity(similarity_matrix, SalesItemCustomerMatrixs, top_n=10):
    CustomerSalesItemMatrixs = csr_matrix(SalesItemCustomerMatrixs.T)
    CustomerSalesItemScores = CustomerSalesItemMatrixs.dot(similarity_matrix) # sum of similarities to all purchased products
    RecForCust = []
    for user_id in range(CustomerSalesItemScores.shape[0]):
        scores = CustomerSalesItemScores[user_id, :]
        purchased_items = CustomerSalesItemMatrixs.indices[CustomerSalesItemMatrixs.indptr[user_id]:
                                                      CustomerSalesItemMatrixs.indptr[user_id+1]]
        scores[purchased_items] = -1 # do not recommend already purchased SalesItem
        top_products_ids = np.argsort(scores)[-top_n:][::-1]

        recommendations = pd.DataFrame(
            top_products_ids.reshape(1, -1),
            index=[user_id],
            columns=['Top%s' % (i+1) for i in range(top_n)])
        RecForCust.append(recommendations)
        
    return pd.concat(RecForCust) 

In [52]:
def get_recommendations(purchase_data):
    # replace Customer and product labels with consecutive integer ids --> not needed any longer
    user_label_encoder = LabelEncoder()
    user_ids = user_label_encoder.fit_transform(purchase_data.Customer)
    
    product_label_encoder = LabelEncoder()
    product_ids = product_label_encoder.fit_transform(purchase_data.SalesItem)

    # compute recommendations
    similarity_matrix, SalesItemCustomerMatrixs = GetItemItemSim(user_ids, product_ids)

    recommendations = get_recommendations_from_similarity(similarity_matrix, SalesItemCustomerMatrixs)

    # project ids back to original labels
    recommendations.index = user_label_encoder.inverse_transform(recommendations.index) #only needed, if you want to map back from codes to names
    
    for i in range(recommendations.shape[1]):
        recommendations.iloc[:, i] = product_label_encoder.inverse_transform(recommendations.iloc[:, i])#only needed, if you want to map back from codes to names
    return recommendations


In [53]:
# this cell will start our recommender
start = timer()
recommendations = get_recommendations(purchase_data)
end = timer()

print('\nRuntime: %0.2fs' % (end - start))


Runtime: 0.50s


In [54]:
print (recommendations)

     Top1   Top2   Top3   Top4   Top5   Top6   Top7   Top8   Top9  Top10
0    I769   I253  I1146  I1138   I749   I613   I752   I756   I430   I750
1   I1207   I857  I1706  I2532  I1456  I1120  I1988  I2482  I1888  I1890
2   I1206   I301   I390  I1254  I1453    I15  I1283   I820  I1446   I799
3     I72  I1206   I795   I205   I156   I229   I226   I655   I965   I107
4    I954    I15   I635   I911   I419   I287  I1301   I861   I859   I284
5    I598   I264   I411   I552   I649   I134   I346   I880  I1014  I2676
6     I72   I419   I795   I229   I655   I500   I226  I1206   I514   I911
7    I795  I1230   I911   I806  I1206  I1254   I965  I1284  I1283   I954
8    I964   I399  I1788    I79   I145   I471    I97   I269   I897   I123
9      I2   I769   I253  I1146  I1138   I752   I430   I753   I431   I749
10  I2763  I2426  I2765  I2090  I2522  I2202  I2413  I2525  I1922  I1627
11  I1230   I100   I614  I1245   I842   I840   I147   I839   I227   I506
12  I1206  I1230   I109   I500  I1284  I1301   I281

In [55]:
dfrec = recommendations

In [56]:
dfrec.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10
0,I769,I253,I1146,I1138,I749,I613,I752,I756,I430,I750
1,I1207,I857,I1706,I2532,I1456,I1120,I1988,I2482,I1888,I1890
2,I1206,I301,I390,I1254,I1453,I15,I1283,I820,I1446,I799
3,I72,I1206,I795,I205,I156,I229,I226,I655,I965,I107
4,I954,I15,I635,I911,I419,I287,I1301,I861,I859,I284


In [57]:
dfrec.to_excel("ExportCustomerName-Itemname.xlsx")

# Evaluation: Coverage

In [58]:
# calculate the share of items recommended
all_recommended_items = recommendations.values.reshape(-1, 1)[:, 0]
n_items = len(np.unique(purchase_data.SalesItem))
n_recommended_items = len(np.unique(all_recommended_items))
coverage = n_recommended_items / n_items

print('Coverage: %0.2f' % coverage)

Coverage: 0.06


In [59]:
abs_rec_frequency = pd.DataFrame({'recommended': all_recommended_items, 'count': 1}).groupby('recommended').count()
top_5_recs_overall = (abs_rec_frequency.sort_values('count', ascending=False) /
                      abs_rec_frequency['count'].sum())[:5]

print('5 most frequent recommendations:\n %s' % top_5_recs_overall)

5 most frequent recommendations:
                 count
recommended          
I1138        0.025714
I1146        0.025714
I1206        0.017143
I769         0.017143
I1142        0.017143


# Turicreate

In [60]:
pip install turicreate

In [61]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

In [62]:
data = pd.read_excel('/content/gdrive/My Drive/DDDDFolder/DDDD.xlsx')
data.head()

,SalesDate,SalesValue,SalesAmount,Customer,SalesTransactionID,SalesItem
0,2018-09-28,8280.0,10,0,0,0
1,2018-09-28,7452.0,10,0,0,0
2,2019-04-23,21114.0,30,0,1,0
3,2019-04-23,7038.0,10,0,1,1
4,2019-04-23,7000.0,2,0,1,2


In [63]:
def create_DataBinary(data):
    DataBinary = data.copy()
    DataBinary['PurchasedYes'] = 1
    return DataBinary

In [64]:
DataBinary = create_DataBinary(data)

In [65]:
DataBinary.head()

,SalesDate,SalesValue,SalesAmount,Customer,SalesTransactionID,SalesItem,PurchasedYes
0,2018-09-28,8280.0,10,0,0,0,1
1,2018-09-28,7452.0,10,0,0,0,1
2,2019-04-23,21114.0,30,0,1,0,1
3,2019-04-23,7038.0,10,0,1,1,1
4,2019-04-23,7000.0,2,0,1,2,1


In [66]:
def normalize_data(data):
    DfMatrix = pd.pivot_table(data, values='SalesAmount', index='Customer', columns='SalesItem')
    DfMatrixNorm = (DfMatrix-DfMatrix.min())/(DfMatrix.max()-DfMatrix.min())
    d = DfMatrixNorm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['Customer'], value_name='scaled_purchase_freq').dropna()

In [67]:
DataNorm=normalize_data(data)
print(DataNorm.shape)
DataNorm.head()

(12674, 3)


,Customer,SalesItem,scaled_purchase_freq
0,0,0,0.377728
9,9,0,0.047214
16,16,0,0.165660
18,18,0,0.037792
22,23,0,0.000000


In [68]:
train, test = train_test_split(data2, test_size = .2)
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)
print(train.shape, test.shape)

(273137, 3) (68285, 3)


In [69]:
# We can define a function for this above step as follows
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data


In [70]:
# lets try with both dummy table and scaled/normalized purchase table
train_DataBinary, test_DataBinary = split_data(DataBinary)
train_DataNorm, test_DataNorm = split_data(DataNorm)

In [71]:
DfResetted.head()

,Customer,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,3695,3696,3697,3698,3699,3700,3701,3702,3703,3704,3720,3721,3722,3723,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741,3742,3743,3744,3745,3746,3747,3748,3750
0,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
ItemCustMatrixTuri = pd.pivot_table(data, values='SalesAmount', index='SalesItem', columns='Customer') #SalesAmount vs PurchasedYes, Customer must be int for Turicreate to work, that is why data is used instead of data2
ItemCustMatrixTuri.head()

Customer,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35
SalesItem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,15.611111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.526316,NaN,NaN,NaN,NaN,NaN,NaN,7.857143,NaN,3.181818,NaN,NaN,NaN,1.8,38.363636,NaN,NaN,NaN,NaN,NaN,3.000000,3.2,NaN,NaN,NaN,7.0
1,8.777778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.166667,NaN,NaN,NaN,18.571429,68.75,47.692308,24.891892,NaN,6.750000,NaN,NaN,NaN,1.5,38.800000,NaN,NaN,NaN,NaN,20.0,1.606299,NaN,NaN,NaN,NaN,NaN
2,1.444444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,1.000000,NaN,NaN,NaN,1.0,4.100000,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,NaN,NaN,NaN,3.0
3,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,3.473684,NaN,1.444444,NaN,NaN,2.666667,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.238095,12.083333,NaN,NaN,1.0,NaN,NaN,NaN,2.1,2.025210,NaN,NaN,NaN,NaN,NaN


In [73]:
ItemCustMatrixTuri=ItemCustMatrixTuri.fillna(0)
DfItemCustMatrix = ItemCustMatrixTuri.reset_index().rename_axis(None, axis=1)
DfItemCustMatrix.head()

,SalesItem,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0,15.611111,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,3.526316,0.0,0.0,0.0,0.000000,0.00,0.000000,7.857143,0.0,3.181818,0.0,0.000000,0.000000,1.8,38.363636,0.0,0.0,0.0,0.0,0.0,3.000000,3.2,0.0,0.0,0.0,7.0
1,1,8.777778,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,6.166667,0.0,0.0,0.0,18.571429,68.75,47.692308,24.891892,0.0,6.750000,0.0,0.000000,0.000000,1.5,38.800000,0.0,0.0,0.0,0.0,20.0,1.606299,0.0,0.0,0.0,0.0,0.0
2,2,1.444444,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,2.000000,0.0,1.000000,0.0,0.000000,0.000000,1.0,4.100000,0.0,0.0,0.0,0.0,0.0,2.666667,0.0,0.0,0.0,0.0,3.0
3,3,0.000000,1.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,27.571429,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,4,0.000000,1.0,3.473684,0.0,1.444444,0.0,0.0,2.666667,2.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,1.238095,12.083333,0.0,0.000000,1.0,0.0,0.0,0.0,2.1,2.025210,0.0,0.0,0.0,0.0,0.0


In [74]:
transactions = pd.read_excel('/content/gdrive/My Drive/DDDDFolder/DDDDCustomer-ItemMatrixImport.xlsx') 
transactions.head()

,Customer,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0,281,0,0,0,0,0,0,0,0,67,0,0,0,0,0,0,165,0,35,0,0,0,18,1266,0,0,0,0,0,6,16,0,0,0,14
1,1,158,0,0,0,0,0,0,0,0,37,0,0,0,130,825,620,921,0,162,0,0,0,3,1940,0,0,0,0,20,204,0,0,0,0,0
2,2,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,4,0,0,0,3,82,0,0,0,0,0,8,0,0,0,0,3
3,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,193,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,3,66,0,26,0,0,8,2,0,0,0,0,0,0,0,0,0,0,0,26,145,0,0,1,0,0,0,21,241,0,0,0,0,0


In [75]:
# variables to define field names
user_id = 'Customer' 
item_id = 'SalesItem'
target = 'PurchasedYes' 
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [76]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    #Pearson Correlation is simply centered cosine similarity. Pearson correlation is usually used if ratings are in common. 
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [77]:
customers=DfCustomerUnique
customers.head()

,Customer
0,0
1,1
2,2
3,3
4,4


Define Models using Turicreate library Before running a more complicated approach such as collaborative filtering, we should run a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity. In this case, we will be using popularity model. A more complicated but common approach to predict purchase items is collaborative filtering. I will discuss more about the popularity model and collaborative filtering in the later section. For now, let’s first define our variables to use in the models:

In [78]:
# variables to define field names
user_id = 'Customer'
item_id = 'SalesItem'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 

In [79]:
train.groupby(by=item_id)['PurchasedYes'].sum().sort_values(ascending=False).head(20) 

SalesItem
I20     2380
I104    2110
I110    1863
I103    1801
I83     1640
I67     1596
I120    1471
I229    1435
I108    1394
I92     1319
I91     1257
I101    1198
I514    1179
I63     1132
I165    1111
I122    1095
I156    1060
I161    1055
I66     1028
I105     998
Name: PurchasedYes, dtype: int64

In [80]:
name = 'cosine'
target = 'PurchasedYes' 
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 273137 observations with 35 users and 3663 items.

Data prepared in: 0.216968s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.38358s

+----------+-----------+----------------------+------+
| Customer | SalesItem |        score         | rank |
+----------+-----------+----------------------+------+
|    0     |    I753   |  0.6333644787470499  |  1   |
|    0     |    I758   |  0.624368409315745   |  2   |
|    0     |    I757   |  0.624368409315745   |  3   |
|    0     |    I430   |  0.5863815546035767  |  4   |
|    0     |   I1137   |  0.5847456256548563  |  5   |
|    0     |   I1138   |  0.5847456256548563  |  6   |
|    0     |   I1139   |  0.5847456256548563  |  7   |
|    0     |    I769   |  0.5847456256548563  |  8   |
|    0     |    I749   |  0.5840431650479635  |  9   |
|    0     |    I754   |  0.5840431650479635  |  10  |
|    1     |   I1612   | 0.21305563973217476  |  1   |
|    1     |   I1588   | 0.21305563973217476  |  2   |
|    1     |   I1551   |  0.206958078756565   |  3   |
|    1     |   I1686   |  0.206958078756565   |  4   |
|    1     |   I1764   |  0.206958078756565   |  5   |
|    1    

In [81]:
name = 'cosine'
target = 'PurchasedYes'
cos_dummy = model(train_DataBinary, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns SalesDate, SalesValue, SalesAmount, SalesTransactionID;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 273137 observations with 35 users and 3658 items.

Data prepared in: 0.171718s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.402777s

+----------+-----------+----------------------+------+
| Customer | SalesItem |        score         | rank |
+----------+-----------+----------------------+------+
|    0     |    758    |  0.6353145837783813  |  1   |
|    0     |    753    |  0.6052374243736267  |  2   |
|    0     |    749    |  0.5942823737859726  |  3   |
|    0     |    769    |  0.5942690372467041  |  4   |
|    0     |    752    |  0.5734167844057083  |  5   |
|    0     |    757    |  0.4764312505722046  |  6   |
|    0     |    253    |  0.4764312505722046  |  7   |
|    0     |    750    |  0.4764312505722046  |  8   |
|    0     |    754    |  0.4764312505722046  |  9   |
|    0     |    613    |  0.4764312505722046  |  10  |
|    1     |    639    |  0.183076993872722   |  1   |
|    1     |    291    | 0.17670882617433867  |  2   |
|    1     |    575    | 0.16910156855980554  |  3   |
|    1     |    493    | 0.16910156855980554  |  4   |
|    1     |    565    | 0.16910156855980554  |  5   |
|    1    

In [82]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_DataNorm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 10139 observations with 34 users and 2641 items.

Data prepared in: 0.017226s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.233357s

+----------+-----------+-----------------------+------+
| Customer | SalesItem |         score         | rank |
+----------+-----------+-----------------------+------+
|    0     |    1142   |  0.14073100686073303  |  1   |
|    0     |    1139   |  0.12070442736148834  |  2   |
|    0     |    1138   |  0.05750127136707306  |  3   |
|    0     |    1148   |  0.02179056406021118  |  4   |
|    0     |    769    |  0.019457414746284485 |  5   |
|    0     |    752    |  0.018182143568992615 |  6   |
|    0     |    431    |  0.014094844460487366 |  7   |
|    0     |    748    |  0.01113981008529663  |  8   |
|    0     |    758    |  0.01040583848953247  |  9   |
|    0     |    1504   |  0.00967833399772644  |  10  |
|    1     |    273    |  0.017142194968003493 |  1   |
|    1     |    898    |  0.016369503277998704 |  2   |
|    1     |    1262   |  0.016028980414072674 |  3   |
|    1     |    485    |  0.015847672254611284 |  4   |
|    1     |    1243   |  0.015839222149971206 |

In [83]:
# create initial callable variables
models_w_counts = [cos]
models_w_dummy = [cos_dummy]
models_w_norm = [cos_norm]

names_w_counts = ['Cosine Similarity on Purchase Counts']
names_w_dummy = ['Cosine Similarity on Purchase Dummy']
names_w_norm = ['Cosine Similarity on Scaled Purchase Counts']

In [84]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Cosine Similarity on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |  0.1333333333333334 | 0.00517311602587607  |
|   2    | 0.10000000000000003 | 0.006645915264929797 |
|   3    | 0.11111111111111112 | 0.01013476484791139  |
|   4    |        0.125        | 0.015112734031922705 |
|   5    | 0.11333333333333334 | 0.015175062720454251 |
|   6    | 0.09444444444444446 | 0.015175062720454251 |
|   7    | 0.09523809523809522 | 0.018263207303463907 |
|   8    |         0.1         | 0.021373081569220184 |
|   9    | 0.09259259259259259 | 0.02180598200212062  |
|   10   | 0.08666666666666668 | 0.02180708222106419  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9503631963134942

Per User RMSE (best)
+----------+------------

In [85]:
eval_dummy = tc.recommender.util.compare_models(test_DataBinary, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Cosine Similarity on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+---------------------+------------------------+
| cutoff |    mean_precision   |      mean_recall       |
+--------+---------------------+------------------------+
|   1    | 0.06666666666666668 | 0.00019835580246414138 |
|   2    | 0.05000000000000001 |  0.006865022469130807  |
|   3    | 0.04444444444444444 |  0.007765923370031709  |
|   4    |         0.05        |  0.007790188479001866  |
|   5    | 0.04666666666666666 |  0.00825315144196483   |
|   6    |         0.05        |  0.009449378725148635  |
|   7    | 0.04285714285714285 |  0.009449378725148635  |
|   8    |         0.05        |  0.01125667744867427   |
|   9    | 0.05185185185185185 |  0.011636661405422405  |
|   10   | 0.05333333333333334 |  0.012538679450010326  |
+--------+---------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9513478126018896

Per User RMSE (bes

In [86]:
eval_norm = tc.recommender.util.compare_models(test_DataNorm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Cosine Similarity on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    | 0.35714285714285715 | 0.01775209210793221 |
|   2    | 0.39285714285714285 |  0.1344762211534898 |
|   3    |  0.380952380952381  | 0.14203054824222844 |
|   4    |  0.3482142857142857 |  0.1677789015832069 |
|   5    | 0.33571428571428574 | 0.21166182125071603 |
|   6    | 0.30357142857142855 | 0.22111427301848088 |
|   7    |  0.2857142857142857 | 0.23993441767713483 |
|   8    | 0.27232142857142855 | 0.25635995098100756 |
|   9    |  0.253968253968254  | 0.25696768301731104 |
|   10   | 0.24642857142857147 | 0.26712662824655997 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.5299481328967095

Per User RMSE (best)
+----------+-------------------

In [87]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(DataBinary), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='PurchasedYes', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns SalesDate, SalesValue, SalesAmount, SalesTransactionID;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 341422 observations with 35 users and 3725 items.

Data prepared in: 0.208029s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.389574s

+----------+-----------+----------------------+------+
| Customer | SalesItem |        score         | rank |
+----------+-----------+----------------------+------+
|    0     |    769    |  0.6447658985853195  |  1   |
|    0     |    759    |  0.6353145837783813  |  2   |
|    0     |    758    |  0.6353145837783813  |  3   |
|    0     |    753    |  0.6193609237670898  |  4   |
|    0     |    757    |  0.5942823737859726  |  5   |
|    0     |    754    |  0.5942823737859726  |  6   |
|    0     |    749    |  0.5942823737859726  |  7   |
|    0     |    752    |  0.5734167844057083  |  8   |
|    0     |    431    |  0.5363822728395462  |  9   |
|    0     |    430    |  0.5363822728395462  |  10  |
|    1     |    1459   | 0.17570708607727625  |  1   |
|    1     |    726    | 0.16014184254520344  |  2   |
|    1     |    653    | 0.15059965061691571  |  3   |
|    1     |    652    | 0.15059965061691571  |  4   |
|    1     |    639    | 0.15059965061691571  |  5   |
|    1    

In [88]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
CustSelec0 = df_rec[df_rec.Customer == 0]
CustSelec0.head(10)

(350, 4)


,Customer,SalesItem,score,rank
0,0,769,0.644766,1
1,0,759,0.635315,2
2,0,758,0.635315,3
3,0,753,0.619361,4
4,0,757,0.594282,5
5,0,754,0.594282,6
6,0,749,0.594282,7
7,0,752,0.573417,8
8,0,431,0.536382,9
9,0,430,0.536382,10


In [89]:
df_rec.head()

,Customer,SalesItem,score,rank
0,0,769,0.644766,1
1,0,759,0.635315,2
2,0,758,0.635315,3
3,0,753,0.619361,4
4,0,757,0.594282,5


In [90]:
from google.colab import files
df_rec.to_excel('df_rec.xlsx') 

In [91]:
from google.colab import files
files.download('df_rec.xlsx')
#will be saved in my personal download files locally

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [92]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['Customer', 'recommendedProducts']].drop_duplicates().sort_values('Customer').set_index('Customer')

In [93]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    if print_csv:
        df_output.to_csv('/content/gdrive/My Drive/TuriCreate/option2_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option2_recommendation.csv'")
    return df_output

In [95]:
df_output = create_output(cos_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head(20)

An output file can be found in 'output' folder with name 'option2_recommendation.csv'
(35, 1)


,recommendedProducts
Customer,
0,769|759|758|753|757|754|749|752|431|430
1,1459|726|653|652|639|575|565|564|1047|1223
2,183|204|206|966|957|394|324|266|480|968
3,74|96|86|159|128|146|193|161|89|185
4,199|167|2204|2409|2237|2449|2422|2434|2206|2196
5,152|146|96|74|154|985|98|13|185|155
6,226|213|277|256|245|242|279|237|230|307
7,907|851|958|955|929|997|1022|50|985|99
8,181|80|208|174|148|145|123|259|399|269


In [96]:
df_output.head()

,recommendedProducts
Customer,
0,769|759|758|753|757|754|749|752|431|430
1,1459|726|653|652|639|575|565|564|1047|1223
2,183|204|206|966|957|394|324|266|480|968
3,74|96|86|159|128|146|193|161|89|185
4,199|167|2204|2409|2237|2449|2422|2434|2206|2196


In [97]:
from google.colab import files
df_output.to_excel('df_output.xlsx')

In [98]:
from google.colab import files
files.download('df_output.xlsx')
#if it should not work right away, than repeat the import above again and it will work

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [99]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [100]:
customer_recomendation(0)

recommendedProducts    769|759|758|753|757|754|749|752|431|430
Name: 0, dtype: object

In [101]:
customer_recomendation(1)

recommendedProducts    1459|726|653|652|639|575|565|564|1047|1223
Name: 1, dtype: object